# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  1.92it/s]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-01-26 20:55:00.757689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中非常重要的一部分。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云和空气。乾卦的卦辞是“元、亨、利、贞”，表示宇宙万物的本源和运行的规律，以及人类行为准则的正确和美满。

乾卦的六爻分别代表着不同的含义，从下往上依次为：初爻，二爻，三爻，四爻，五爻，上爻。每一爻的阳爻代表阳刚之气，阴爻代表阴柔之气。通过六爻的变化组合，可以得出不同的卦象，例如乾、坤、震、巽、坎、离、艮、兑等。

乾卦主要阐述阳刚之气、力量、创造、领导等象征。在易经中，乾卦与坤卦被认为是天地的两极，象征着宇宙万物的形成和运行。乾卦也常常被用来象征父亲、领导、丈夫等阳刚之气强烈的角色，以及权力和威严等。同时，乾卦还代表着天空、云彩、树木等，具有开放、广博、包容的特点。


In [9]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的一种卦象，由两个阴爻夹一个阳爻构成，象征着地水之合。这个卦象主要阐述柔顺、适应、团结、合作等象征。

在《易经》中，师卦的卦辞是“西南得朋，东北丧朋。安贞，吉”，表示通过柔顺、适应、团结、合作等方法可以获得吉祥。师卦主要代表教师、领导者、导师等角色，以及知识、智慧、技能等。

地水师卦通常被认为是一种中庸之道，具有灵活性和适应性。它告诉我们，在面对不同的环境和挑战时，应该根据实际情况灵活应对，保持柔顺和适应性，同时要善于团结和合作，才能获得吉祥和成功。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [16]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个阳爻（代表阳）的卦象叠加而成，象征万事万物的变化和天象的运行。乾卦包含着刚健强劲的特性，以及天动时行的力量。在周易中，它预示着事物的发展变化趋势，以及人们应该遵循的道德规范。

乾卦的卦象是：
上卦：乾（天），刚健强劲
下卦：坤（地），柔顺 sil

在周易中，乾卦代表的是刚健强劲的特性，以及天动时行的力量。它象征着事物的发展变化趋势，以及人们应该遵循的道德规范。

乾卦的哲学内涵是：
- 卦象刚健强劲
- 天动时行
- 健硬刚强
- 预测事物发展变化趋势
- 人们应该遵循的道德规范

乾卦的运势指数是：
- 事业：健硬刚强，注重力量，努力奋斗
- 感情：刚柔相济，注重和谐
- 健康：刚硬刚强，注重保养
- 投资：健硬刚强，寻求力量
- 事业：健硬刚强，注重力量，努力奋斗
- 感情：刚柔相济，注重和谐
- 健康：刚硬刚强，注重保养

乾卦的运势特征是：
- 事业：健硬刚强，注重力量，努力奋斗
- 感情：刚柔相济，注重和谐
- 健康：刚硬刚强，注重保养
- 投资：健硬刚强，寻求力量
- 事业：健硬刚强，注重力量，努力奋斗
- 感情：刚柔相济，注重和谐
- 健康：刚硬刚强，注重保养

乾卦的装束是：
- 衣衣：穿着刚硬刚强的衣服
- 颜色：使用白色
- 饮食：选择刚硬刚强的食物
- 旅行：去天动时行的地方

乾卦的时运是：
- 好运：健硬刚强，注重力量，努力奋斗
- 倒霉：刚硬刚强，注重力量，努力奋斗


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征刚健强劲的特性。它由下至上阳爻阳阴相间，形成一个阳刚之卦。乾卦的卦辞是“元、亨、利、贞”，预示着天行健健，文武艺高，先武后文，利养元善，志意刚强，纯真无邪。

乾卦的初卦为乾，代表天，象征刚健强劲的特性。第二卦为坤，代表地，象征柔顺，依赖刚健的特性。第三卦为震，代表雷，象征声响巨大，声东击西，利养元善。第四卦为巽，代表风，象征柔顺，依赖刚健的特性。第五卦为坎，代表水，象征刚强，柔顺的特性。第六卦为离，代表火，象征刚强，利养元善。

乾卦的特性是刚健强劲，代表积极，主动，勇于担当，勇于进取，具有领导力。在周易中，乾卦代表天，象征刚健强劲的特性，预示着文武艺

In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由坎卦（水）和坤卦（地）组成的卦象，代表major水地卦。在周易中，这个卦象代表着重大地现象，象征重大地变化。根据《象辞》，这个卦象的卦象是：下坤上坎。根据《断易天机》一书的解释，地水相克，上下相吸，象征的是刚柔相济，众阴承刚。根据context，地水师卦被认为代表一对夫妻相互离异，或者夫妻之间出现矛盾，需要双方都象征硬汉，努力地拖住对方，以力相抗。

虽然sourceof以上内容无法确定是否准确，但是地水师卦被认为是一个象征重大地变化和刚柔相济的卦象，离异双方的努力和坚持可能会带来积极的结果。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三个卦象组成的卦卦，代表田地中的水。它由坎卦（水）和坤卦（地）组成，其卦象为：上坎下坤。卦象解释：坎卦为水，代表田地中的水；坤卦为地，代表田地中的水。 both卦 are known for their calming effect，and are associated with the concept of the earth and water.

堪言卦（卦） 
 堪言卦是卦，是卦象，是预测的卦象，代表田地中的水。它由坎卦（水）和坤卦（地）组成，其卦象为：上坎下坤。卦象解释：坎卦为水，代表田地中的水；坤卦为地，代表田地中的水。 both卦 are known for their calming effect，and are associated with the concept of the earth and water.


In [18]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
{'name': '卦象为：上坎下离，离为离，加卦为加', 'content': '卦象解释：上坎下离，离为离，加卦为加。离，火；坎，水。离为离，指数为3，为刚柔相离的变化。加，加卦，为加，为利，为胜。卦象代表的是刚柔相离的变化，象征的是离为离，离为离，加卦为加。离，火；坎，水。离为离，指数为3，为刚柔相离的变化。加，加卦为利，为胜。\n\n推演卦象：利胜\n\n推演卦象为：离为离，离为离，加卦为加。离，火；坎，水。离为离，指数为3，为刚柔相离的变化。加，加卦为利，为胜。\n\n象辞卦象：火补丁火，互为离火，为刚柔相离的变化。加，为胜。\n\n conclusions\n\n结论1：离为离，为刚柔相离的变化。加，为胜。\n\n结论2：利胜\n\n所以，天水讼卦的卦象为利胜。'}


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦

卦象为：天（乾为天）*地（坤为地）*风（巽为风）*雷（震为雷）*雨（巽为雨）*夏（离为夏）*秋（离为秋）*冬（离为冬）*春（离为春）

卦象解释：

* 天为阳，地为阴。
* 乾为天，为夫为君，为父为孟
* 坤为地，为母为季
* 坤为地, 动而地益成，为无极而起为有
* 震为雷，动而观其象，为带电的雷
* 雨为巽，为云，为夏为显，为柔弱
* 离为夏，为火，为光，为丽
* 丽为秋，为商为众，为丽为美
* 离为冬，为寒为清，为商为限

* 春为离，为寒为温，为开

卦象的含义是：

* 天为阳，地为阴。
* 乾为天，为夫为君，为父为孟
* 坤为地，为母为季
* 坤为地, 动而地益成，为无极而起为有
* 震为雷，动而观其象，为带电的雷
* 雨为巽，为云，为夏为显，为柔弱
* 离为夏，为火，为光，为丽
* 丽为秋，为商为众，为丽为美
* 离为冬，为寒为清，为商为限
* 春为离，为寒为温，为开

* 总体来说，这是一个充满变化和发展的卦象。
* 适合寻求成功和解决冲突。
* 预示吉祥，但需谨慎
* 预示困难，但需耐心。
* 预示成功和发展，但需努力
* 预示困难和高潮，但需谨慎
* 预示成功和发展，但需努力
* 预示成功和发展，但需努力

* 该卦象代表的是：天为阳，地为阴。乾为天，为夫为君，为父为孟


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [19]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [20]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [21]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，具有刚健的象征意义。它由两个阳爻（象征天）叠加而成，代表着刚强、积极、勇敢的特性。在卜卦时，乾卦预示着吉祥如意，但 also需要警惕过于刚强导致的困难。在卜卦中，阳爻代表刚强，阴爻代表柔弱。因此，在决策时应当考虑到事物的两面性，以及刚强可能会导致的困难。

乾卦的卦象是：初九，九二，九三，九四，九五，九六。在每一卦中，阳爻逐渐增加，代表着逐渐刚强、积极、勇敢的特性。

乾卦的核心哲学是：刚强就能成功。但刚强又有极点，超出刚强就会发生灾难。因此，决策时应当考虑到事物的两面性，以及刚强可能会导致的困难。

乾卦的取象术是：取天（阳）为象，代表刚强、积极、勇敢的特性。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，具有刚健的象征意义。它由两个卦象叠加而成，上卦为乾，下卦为坤。上卦的卦象为天，下卦的卦象为地，象征着天地的相应。

乾卦的卦象是三个阳爻（表示天），两个阴爻（表示地），象征着天地的相应。阳爻代表刚健，阴爻代表柔顺。因此，乾卦意味着刚健健忘，具有刚健的力量，但也有柔顺的特性。

在占卜中，乾卦预示着天（刚健）欲往地（柔顺）求得的情况。对于卜者来说，这预示着必须刚健很强，具有刚健的力量，但也要注意保持柔顺，谨慎行事，以免招致困难。

在解卦中，乾卦代表天，具有刚健的象征意义。它预示着刚健健忘，具有刚健的力量，但也有柔顺的特性。在决策时，刚健的特性会使人更容易冲动，因此，应该刚中有柔，注重柔顺，谨慎行事，以免招致困难。


In [22]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由地卦（坤）和坎卦（水）组成，代表地面上的水。地水师卦代表的是指挥官（地）和水的混合，象征着混合着在地上和水上的军队。在占卜时，地面上的水（代表军队）流动，训练着军队，优先考虑攻击方向。

地水师卦的吉祥 outcomes包括：

1. 地面平滑如水（吉祥）， 
2. 军队和平， 
3. 敌军受困， 
4. 战争平息， 
5. 军队安静。

不利因素包括：

1. 地面凹凸不平（ unfinished ground），
2. 河流泛滥（ water over flow），
3. 军队不听命令（ unfaithful army）。

在分析地水师卦时，需要考虑到地面和水流的特征，以及军队的状况和命令。通过对卦象的解读，可以揭示出当前情况的特点，从而得出合适的决策。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由上卦坤（地）和下卦坎（水）组成的卦象，上卦为坤，下卦为坎。在卜筮时，上卦表示地，下卦表示水，象征着地载水的情况。地水师卦的卦象表示了地水相融合的情况，预示着顺利，为吉兆。

地水师卦的哲学含义是：悦于水之清，悦于地之平。悦于水之静，悦于地之静。悦于水之德，悦于地之德。悦于水之志，悦于地之志。悦于水之节，悦于地之节。悦于水之诚，悦于地之诚。

地水师卦的运势：

- 0年：利于土地，安宅落业。
- 1年：利于涉水，涉远涉深。
- 2年：利于土地，利于 home。
- 3年：利于涉水，利于远行。
- 4年：利于土地，利于 home。
- 5年：利于涉水，利于远行。
- 6年：利于土地，安宅落业。
- 7年：利于涉水，利于 home。
- 8年：利于土地，利于 home。
- 9年：利于涉水，利于远行。
- 10年：利于土地，利于 home。

地水师卦的事业和经商预测：

- 适宜的职业：土地、房屋、农业、水利等。
- 不适宜的职业：水上、航行、远行、冒险等。
- 适合的时机：稳定、平静、温和、和谐等。
- 不适宜的时机：变动、冲突、艰难、危险等。

总体来说，地水师卦预示着顺利，为吉兆，但需要稳定、温和、和谐的处理水边事物。


In [23]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


ChatGLM3-6B微调后：
[gMASK]sop 周易中的讼卦是什么卦象？ 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [25]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240118_164514"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [26]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [27]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，具有刚健的象征意义。它由两个卦象叠加而成，上卦是乾卦，下卦是坤卦。在卜问中，乾卦预示着天象的运行，预示着吉利的出现。在占卜中，大吉的结论出现了，说明吉利将会长期持续。然而，在战争中，吉利将会转化为不利的情况，带来军队的失败。

乾卦的卦象是两个阳爻（象征天）叠加而成，具有强烈的阳刚之气。这种卦象象征着刚健、强盛、勇敢和力量。在解卦中，刚健的刚象出现了，预示着不久的将来，刚健的刚象将会出现，带来积极的影响。

乾卦的卦辞这样描述：“元、亨、利、贞”，意味着吉祥、正谋、顺利、正道。这一卦显示了天象的运行，预示着吉利将会出现，具有积极的意义。

总体而言，乾卦预示着天象的运行，带来吉利和力量，预示着不久的将来，刚健的刚象将会出现，带来积极的影响。在解卦中，刚健的刚象出现了，预示着不久的将来，刚健的刚象将会出现，带来积极的影响。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，具有刚健的象征意义。它由两个卦象叠加而成，上卦为乾，下卦为坤。上卦的卦象为天，下卦的卦象为地，象征着天地的相应。

乾卦的卦象是三个阳爻（表示天），两个阴爻（表示地），象征着天地的相应。阳爻代表刚健，阴爻代表柔顺。因此，乾卦意味着刚健健忘，具有刚健的力量，但也有柔顺的特性。

在占卜中，乾卦预示着天（刚健）欲往地（柔顺）求得的情况。对于卜者来说，这预示着必须刚健很强，具有刚健的力量，但也要注意保持柔顺，谨慎行事，以免招致困难。

在解卦中，乾卦代表天，具有刚健的象征意义。它预示着刚健健忘，具有刚健的力量，但也有柔顺的特性。在决策时，刚健的特性会使人更容易冲动，因此，应该刚中有柔，注重柔顺，谨慎行事，以免招致困难。


In [ ]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

In [ ]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")